We want to copy the weights for the HiFiGAN from the XTTS model and check that our model spits out the same thing as a sanity check

We first download TTS, and the model checkpoint for XTTS.

In [ ]:
!wget "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"
!wget "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/config.json"

/Users/tugdual/miniconda3/envs/xtts/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-11-09 18:59:16--  https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth
Resolving coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)... 3.67.33.93, 3.77.103.135
Connecting to coqui.gateway.scarf.sh (coqui.gateway.scarf.sh)|3.67.33.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth [following]
--2024-11-09 18:59:16--  https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth
Resolving huggingface.co (huggingface.co)... 2600:9000:244f:f600:17:b174:6d00:93a1, 2600:9000:244f:3600:17:b174:6d00:93a1, 2600:9000:244f:e200:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:244f:f600:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3e/73/3e73be38940b0ce7d10274a3412defcd0d3c9ea879d08bc8761d46cf6bbde3fa/c7ea20001c6a0a841c77e252d8409f6a74fb423e79b3206a0771ba5989776187?response-content-di

In [ ]:
!git clone git@github.com:nguyenhoanganh2002/XTTSv2-Finetuning-for-New-Languages.git
!mv XTTSv2-Finetuning-for-New-Languages/* .

We should now be able to load the model as it is used in the XTTSv2.

In [21]:
from TTS.tts.layers.xtts.dvae import DiscreteVAE
import jax
import torch
import equinox as eqx
import pprint as pp

# Exported from .ipynb using python3 export.py (Exports all cells with the export tag)
from TTS.tts.layers.xtts.trainer.gpt_trainer import (
    GPTArgs,
    GPTTrainerConfig,
    GPTTrainer,
    XttsAudioConfig,
)
from TTS.tts.models.xtts import Xtts
from ResNet_for_XTTS import ResNet

# init args and config
model_args = GPTArgs(
    max_conditioning_length=132300,  # 6 secs
    min_conditioning_length=11025,  # 0.5 secs
    debug_loading_failures=False,
    max_wav_length=255995,  # ~11.6 seconds
    max_text_length=200,
    mel_norm_file="./mel_stats",
    dvae_checkpoint="",
    xtts_checkpoint="./model.pth",  # checkpoint path of the model that you want to fine-tune
    tokenizer_file="",
    gpt_num_audio_tokens=1026,
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
)
audio_config = XttsAudioConfig(
    sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000
)

config = GPTTrainerConfig()

config.load_json("./config.json")

config.epochs = 1
config.model_args = model_args
config.audio = audio_config
config.num_loader_workers = 8
config.eval_split_max_size = 256
config.print_step = 50
config.plot_step = 100
config.log_model_step = 100
config.save_n_checkpoints = 1
config.save_checkpoints = True
config.print_eval = False
config.optimizer = "AdamW"
config.optimizer_params = {"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1}
config.test_sentences = []

their_resnet = Xtts(config).hifigan_decoder.speaker_encoder
# init the model from config

# Take all the params for the torch model. Print them and map them to our model.

torch_params = {
    name: param.detach().numpy() for name, param in their_resnet.named_parameters()
}

# for x in torch_params:
#     print(x)
#     print(torch_params[x].shape)
# # print(v)


def boop(path, a):
    seq = [str(p).strip("[].") for p in path]
    print(".".join(seq))
    try:
        print(a.shape)
    except:
        print(a)


key = jax.random.PRNGKey(2)

our_resnet, state = eqx.nn.make_with_state(ResNet)(64, 512, key=key)
test = jax.tree_util.tree_map_with_path(lambda x, a: boop(x, a), our_resnet)

conv1.weight
(32, 1, 3, 3)
conv1.bias
(32, 1, 1)
batch_norm.weight
(32,)
batch_norm.bias
(32,)
batch_norm.axis_name
batch
batch_norm.inference
False
layer1.0.conv1.weight
(32, 32, 3, 3)
layer1.0.conv2.weight
(32, 32, 3, 3)
layer1.0.bn1.weight
(32,)
layer1.0.bn1.bias
(32,)
layer1.0.bn1.axis_name
batch
layer1.0.bn1.inference
False
layer1.0.bn2.weight
(32,)
layer1.0.bn2.bias
(32,)
layer1.0.bn2.axis_name
batch
layer1.0.bn2.inference
False
layer1.0.se.fc1.weight
(4, 32)
layer1.0.se.fc1.bias
(4,)
layer1.0.se.fc2.weight
(32, 4)
layer1.0.se.fc2.bias
(32,)
layer1.1.conv1.weight
(32, 32, 3, 3)
layer1.1.conv2.weight
(32, 32, 3, 3)
layer1.1.bn1.weight
(32,)
layer1.1.bn1.bias
(32,)
layer1.1.bn1.axis_name
batch
layer1.1.bn1.inference
False
layer1.1.bn2.weight
(32,)
layer1.1.bn2.bias
(32,)
layer1.1.bn2.axis_name
batch
layer1.1.bn2.inference
False
layer1.1.se.fc1.weight
(4, 32)
layer1.1.se.fc1.bias
(4,)
layer1.1.se.fc2.weight
(32, 4)
layer1.1.se.fc2.bias
(32,)
layer1.2.conv1.weight
(32, 32, 3, 3)
laye

In [28]:
torch_to_jax_keys = {
    ("batch_norm.weight", "bn1.weight"),
    ("batch_norm.bias", "bn1.bias"),
    ("attention_conv1.bias", "attention.0.bias"),
    ("attention_conv1.weight", "attention.0.weight"),
    ("attention_batch_norm.bias", "attention.2.bias"),
    ("attention_batch_norm.weight", "attention.2.weight"),
    ("attention_conv2.bias", "attention.3.bias"),
    ("attention_conv2.weight", "attention.3.weight"),
}


# Function to update the JAX model parameters
def update_params(path, x):
    seq = [str(p).strip("[].") for p in path]
    # print(seq)
    stri = ".".join(seq)
    for jax_key, torch_key in torch_to_jax_keys:
        if jax_key == stri:
            if "batch_norm" in stri:
                return jax.numpy.array(torch_params[torch_key])
            if "bias" in jax_key:
                return jax.numpy.expand_dims(torch_params[torch_key], -1)
            return jax.numpy.array(torch_params[torch_key])
    if stri in torch_params.keys():
        if "bias" in seq:
            # if "fc.bias" == stri:
            #     return jax.numpy.array(torch_params[stri])
            if "conv1" in seq:
                return jax.numpy.expand_dims(
                    jax.numpy.expand_dims(torch_params[stri], -1), -1
                )
            return jax.numpy.array(torch_params[stri])
        if "fc" in stri:
            return jax.numpy.array(torch_params[stri])
        return jax.numpy.array(torch_params[stri])
    if len(seq) <= 2:
        return x
    if "fc1" in seq[3]:
        if seq[-1] == "weight":
            return jax.numpy.array(
                torch_params[
                    ".".join(
                        [
                            seq[0],
                            seq[1],
                            seq[2],
                            "fc.0",
                            "weight",
                        ]
                    )
                ]
            )
        elif seq[-1] == "bias":
            return jax.numpy.array(
                torch_params[
                    ".".join(
                        [
                            seq[0],
                            seq[1],
                            seq[2],
                            "fc.0",
                            "bias",
                        ]
                    )
                ]
            )
    if "fc2" in seq[3]:
        if seq[-1] == "weight":
            return jax.numpy.array(
                torch_params[
                    ".".join(
                        [
                            seq[0],
                            seq[1],
                            seq[2],
                            "fc.2",
                            "weight",
                        ]
                    )
                ]
            )
        elif seq[-1] == "bias":
            return jax.numpy.array(
                torch_params[
                    ".".join(
                        [
                            seq[0],
                            seq[1],
                            seq[2],
                            "fc.2",
                            "bias",
                        ]
                    )
                ]
            )
    if seq[2] == "downsample":
        if seq[-1] == "weight":
            return jax.numpy.array(
                torch_params[
                    ".".join(
                        [
                            seq[0],
                            seq[1],
                            seq[2],
                            seq[4],
                            "weight",
                        ]
                    )
                ]
            )
        elif seq[-1] == "bias":
            return jax.numpy.array(
                torch_params[
                    ".".join(
                        [
                            seq[0],
                            seq[1],
                            seq[2],
                            seq[4],
                            "bias",
                        ]
                    )
                ]
            )
    return x


# Grab the checky ones that don't have a name but that we need
# torch_params["dvae.codebook.embed"] = dvae.codebook.embed
# dvae.load_state_dict(torch.load(dvae_pretrained), strict=False)
# print(list(dvae.decoder.state_dict()))
# Updatorche the JAX model parameters
our_resnet = jax.tree_util.tree_map_with_path(update_params, our_resnet)

eqx.tree_serialise_leaves("./xttsresnet.eqx", our_resnet)
# Replace the encoder in the model with the updated encoder
# model = eqx.tree_at(lambda m: m, model, model)
# print(model.quantizer.codebook.shape)
# eqx.tree_serialise_leaves("./xttsvqvae.eqx", model)

Below are chaotic tests I made to check things worked, progressively going through the model.

In [19]:
import numpy as np
import torch
import jax

x = torch.ones((1, 64, 4))
x1 = jax.numpy.array(x.numpy())

# their_resnet = their_resnet.eval()
# their_resnet = Xtts(config).hifigan_decoder.speaker_encoder
our_resnet, state = eqx.nn.make_with_state(ResNet)(64, 1, key=key)
our_resnet = jax.tree_util.tree_map_with_path(update_params, our_resnet)
# our_resnet = eqx.nn.inference_mode(our_resnet)

torch.testing.assert_close(x, torch.from_numpy(np.array(x1)))
# For the possible audio input instaed of mel
y = x.squeeze_(1)
print(y.min())

y = (y + 1e-6).log()
print(y)
# y = their_resnet.instancenorm(y).unsqueeze(1)
# y = their_resnet.conv1(y)
# y = their_resnet.relu(y)
# y = their_resnet.bn1(y)
# y = their_resnet.layer1(y)
# y = their_resnet.layer2(y)
# y = their_resnet.layer3(y)
# y = their_resnet.layer4(y)
# y += their_resnet.layer1[0].downsample(y)

# y = their_resnet.layer1[0].se(y)
print(x1.shape)
print(np.min(x1))
x1 = jax.lax.clamp(min=float(0), x=x1, max=jax.numpy.inf)
print(np.min(x1))

y1 = jax.numpy.log(x1 + 1e-6)
print(y1)
# y1 = jax.vmap(eqx.nn.GroupNorm(64, 64, channelwise_affine=False))(y1)
# y1 = jax.numpy.expand_dims(y1, 1)
# print(y1.shape)
# y1 = jax.vmap(our_resnet.conv1)(y1)
# print(y1.shape)
# y1 = jax.nn.relu(y1)
# y1, state = jax.vmap(
#     our_resnet.batch_norm, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
# )(y1, state)
# for layer in our_resnet.layer1:
#     y1, state = jax.vmap(
#         layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
#     )(y1, state)
# for layer in our_resnet.layer2:
#     y1, state = jax.vmap(
#         layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
#     )(y1, state)
# for layer in our_resnet.layer3:
#     y1, state = jax.vmap(
#         layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
#     )(y1, state)
# for layer in our_resnet.layer4:
#     y1, state = jax.vmap(
#         layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
#     )(y1, state)
# y1 = jax.vmap(our_resnet.layer1[0].conv1)(z1)
# y1 = jax.nn.relu(y1)
# y1, state = jax.vmap(
#     our_resnet.layer1[0].bn1, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
# )(y1, state)
# y1 = jax.vmap(our_resnet.layer1[0].conv2)(y1)
# y1, state = jax.vmap(
#     our_resnet.layer1[0].bn2, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
# )(y1, state)
# y1 = jax.vmap(our_resnet.layer1[0].se)(y1)
# # y1 += jax.vmap(our_resnet.layer1[0].downsample)(y1)
# y1 += z1
# y1 = jax.nn.relu(y1)

# def mul(x, y):
#     return x * y


# y1 = jax.vmap(mul)(z, y1)
# y1 = jax.vmap(our_resnet.layer1[0].se.)
# y1 = jax.vmap(our_resnet.layer1[0].se)(y1)


torch.testing.assert_close(y, torch.from_numpy(np.array(y1)))

tensor(1.)
tensor([[[9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07, 9.5367e-07],
         [9.5367e-07, 9.5367e-07, 9.5367e-07,

Although there still is a difference, it's low enough that we musn't waste too much time on it. I suspect it comes from the bias at the last layer conv...

In [ ]:
import numpy as np
import torch
import jax

x = torch.randn((1, 256, 8, 2))
y = jax.numpy.array(x.numpy())[0]

# their_resnet = their_resnet.eval()
# their_resnet = Xtts(config).hifigan_decoder.speaker_encoder
our_resnet, state = eqx.nn.make_with_state(ResNet)(64, 512, key=key)
our_resnet = jax.tree_util.tree_map_with_path(update_params, our_resnet)

x = x.reshape(x.size()[0], -1, x.size()[-1])
print(x.shape)
w = their_resnet.attention(x)
print(w.shape)

mu = torch.sum(x * w, dim=2)
pouet = (torch.sum((x**2) * w, dim=2) - mu**2).clamp(min=1e-5)
sg = torch.sqrt(pouet)
x = torch.cat((mu, sg), 1)
x = their_resnet.fc(x)
print(x.shape)
y = jax.numpy.reshape(y, (-1, y.shape[-1]))
# print(y.shape)

# TODO not really justified...
v = our_resnet.attention_conv1(y)
v = jax.nn.relu(v)
v = jax.numpy.expand_dims(v, 0)
v, state = jax.vmap(
    our_resnet.attention_batch_norm,
    axis_name="batch",
    in_axes=(0, None),
    out_axes=(0, None),
)(v, state)
v = v[0]
v = our_resnet.attention_conv2(v)  # W represents the
v = jax.nn.softmax(v, axis=1)
mu2 = jax.numpy.sum(y * v, axis=1)
pouet2 = jax.lax.clamp(
    min=1e-5, x=jax.numpy.sum((y**2) * v, axis=1) - mu2**2, max=jax.numpy.inf
)
torch.testing.assert_close(pouet, torch.from_numpy(np.array(np.expand_dims(pouet2, 0))))
sg2 = jax.numpy.sqrt(pouet2)
y = jax.lax.concatenate((mu2, sg2), 0)

print(y.shape)
print(our_resnet.fc.weight.shape)
y = our_resnet.fc(y)
y = jax.numpy.expand_dims(y, 0)
print(y.shape)

torch.testing.assert_close(x, torch.from_numpy(np.array(y)))

torch.Size([1, 2048, 2])
torch.Size([1, 2048, 2])
torch.Size([1, 512])
(4096,)
(512, 4096)
(1, 512)


In [10]:
import numpy as np
import torch
import jax

x = torch.randn((1, 64, 40))
x1 = jax.numpy.array(x.numpy())

# their_resnet = their_resnet.eval()
# their_resnet = Xtts(config).hifigan_decoder.speaker_encoder
our_resnet, state = eqx.nn.make_with_state(ResNet)(64, 1, key=key)
our_resnet = jax.tree_util.tree_map_with_path(update_params, our_resnet)
# our_resnet = eqx.nn.inference_mode(our_resnet)

torch.testing.assert_close(x, torch.from_numpy(np.array(x1)))
# For the possible audio input instaed of mel
y = x.squeeze_(1)
y = their_resnet.instancenorm(y).unsqueeze(1)
y = their_resnet.conv1(y)
y = their_resnet.relu(y)
y = their_resnet.bn1(y)
y = their_resnet.layer1(y)
y = their_resnet.layer2(y)
y = their_resnet.layer3(y)
x = their_resnet.layer4(y)
x = x.reshape(x.size()[0], -1, x.size()[-1])
w = their_resnet.attention(x)

mu = torch.sum(x * w, dim=2)
sg = torch.sqrt((torch.sum((x**2) * w, dim=2) - mu**2).clamp(min=1e-5))
x = torch.cat((mu, sg), 1)
x = their_resnet.fc(x)
# y += their_resnet.layer1[0].downsample(y)

# y = their_resnet.layer1[0].se(y)
y1 = jax.vmap(eqx.nn.GroupNorm(64, 64, channelwise_affine=False))(x1)
y1 = jax.numpy.expand_dims(y1, 1)
y1 = jax.vmap(our_resnet.conv1)(y1)
y1 = jax.nn.relu(y1)
y1, state = jax.vmap(
    our_resnet.batch_norm, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
)(y1, state)
for layer in our_resnet.layer1:
    y1, state = jax.vmap(
        layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
    )(y1, state)
for layer in our_resnet.layer2:
    y1, state = jax.vmap(
        layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
    )(y1, state)
for layer in our_resnet.layer3:
    y1, state = jax.vmap(
        layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
    )(y1, state)
for layer in our_resnet.layer4:
    y1, state = jax.vmap(
        layer, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
    )(y1, state)

y = y1
y = jax.numpy.reshape(y, (-1, y.shape[-1]))
v = our_resnet.attention_conv1(y)
v = jax.nn.relu(v)
v = jax.numpy.expand_dims(v, 0)
v, state = jax.vmap(
    our_resnet.attention_batch_norm,
    axis_name="batch",
    in_axes=(0, None),
    out_axes=(0, None),
)(v, state)
v = v[0]
v = our_resnet.attention_conv2(v)  # W represents the
v = jax.nn.softmax(v, axis=1)
mu2 = jax.numpy.sum(y * v, axis=1)
sg2 = jax.numpy.sqrt(
    jax.lax.clamp(
        min=1e-5, x=jax.numpy.sum((y**2) * v, axis=1) - mu2**2, max=jax.numpy.inf
    )
)
y = jax.lax.concatenate((mu2, sg2), 0)

y = our_resnet.fc(y)
y = jax.numpy.expand_dims(y, 0)

torch.testing.assert_close(x, torch.from_numpy(np.array(y)))

AssertionError: Tensor-likes are not close!

Mismatched elements: 251 / 512 (49.0%)
Greatest absolute difference: 4.2222440242767334e-05 at index (0, 459) (up to 1e-05 allowed)
Greatest relative difference: 0.008624270558357239 at index (0, 387) (up to 1.3e-06 allowed)

In [4]:
their_resnet.log_input

True

In [11]:
import numpy as np
import torch
import jax

x = torch.randn((1, 256, 8, 2))
y = jax.numpy.array(x.numpy())[0]
our_resnet, state = eqx.nn.make_with_state(ResNet)(64, 512, key=key)
our_resnet = jax.tree_util.tree_map_with_path(update_params, our_resnet)

x = x.reshape(x.size()[0], -1, x.size()[-1])

w = their_resnet.attention(x)

if their_resnet.encoder_type == "SAP":
    x = torch.sum(x * w, dim=2)
elif their_resnet.encoder_type == "ASP":
    mu = torch.sum(x * w, dim=2)
    sg = torch.sqrt((torch.sum((x**2) * w, dim=2) - mu**2).clamp(min=1e-5))
    x = torch.cat((mu, sg), 1)

x = x.view(x.size()[0], -1)
x = their_resnet.fc(x)

y = jax.numpy.reshape(y, (-1, y.shape[-1]))
# TODO not really justified...
w = our_resnet.attention_conv1(y)
w = jax.nn.relu(w)
w = np.expand_dims(w, 0)
w, state = jax.vmap(
    our_resnet.attention_batch_norm,
    axis_name="batch",
    in_axes=(0, None),
    out_axes=(0, None),
)(w, state)
w = w[0]
w = our_resnet.attention_conv2(w)  # W represents the
w = jax.nn.softmax(w, axis=1)

mu = jax.numpy.sum(y * w, axis=1)
sg = jax.lax.clamp(
    min=1e-5,
    x=jax.numpy.sum((y**2) * w, axis=1) - mu**2,
    max=jax.numpy.float32(10),
)
sg = jax.numpy.sqrt(sg)

y = jax.lax.concatenate((mu, sg), 0)

y = our_resnet.fc(y)

y = np.expand_dims(y, 0)

torch.testing.assert_close(x, torch.from_numpy(np.array(y)))

In [3]:
import numpy as np
import torch
import jax

# x = torch.randn((1, 256, 8, 2))
x = torch.randn((1, 64, 80))
x1 = jax.numpy.array(x.numpy())

our_resnet, state = eqx.nn.make_with_state(ResNet)(64, 512, key=key)
our_resnet = jax.tree_util.tree_map_with_path(update_params, our_resnet)

their_resnet.log_input = False
their_resnet.use_torch_spec = False
y = their_resnet(x)
x1 = jax.numpy.expand_dims(x1, 0)
y1, state = jax.vmap(
    our_resnet, axis_name="batch", in_axes=(0, None), out_axes=(0, None)
)(x1, state)
y1 = y1
torch.testing.assert_close(y, torch.from_numpy(np.array(y1)))

AssertionError: Tensor-likes are not close!

Mismatched elements: 15 / 512 (2.9%)
Greatest absolute difference: 1.329183578491211e-05 at index (0, 472) (up to 1e-05 allowed)
Greatest relative difference: 0.0002463693672325462 at index (0, 183) (up to 1.3e-06 allowed)

In [27]:
from ResNet_for_XTTS import ResNet


model, state = eqx.nn.make_with_state(ResNet)(64, 512, key=jax.random.PRNGKey(1))

model = eqx.tree_deserialise_leaves("./xttsresnet.eqx", model)

RuntimeError: Deserialised leaf at path (GetAttrKey(name='layer1'), SequenceKey(idx=0), GetAttrKey(name='bn1'), GetAttrKey(name='bias')) has changed shape from (32,) in `like` to (32, 1) on disk.